# AutoGluon Multimodal with Deep Learning Containers on SageMaker

[AutoGluon](https://github.com/awslabs/autogluon) automates machine learning tasks enabling you to easily achieve strong predictive performance in your applications. With just a few lines of code, you can train and deploy high-accuracy deep learning models on tabular, image, and text data.
This example shows how to use AutoGluon-Multimodal with Amazon SageMaker by applying [pre-built deep learning containers](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#autogluon-training-containers).

# Prerequisites

In [ ]:
!python -m pip install --upgrade pip
!python -m pip install -U 'sagemaker>=2.103.0'

In [ ]:
import base64
import os
import shutil
import urllib
import zipfile

import boto3
import sagemaker
import pandas as pd
import numpy as np
from sagemaker import utils
from sagemaker.serializers import CSVSerializer

from ag_model import (
    AutoGluonTraining,
    AutoGluonInferenceModel,
    AutoGluonMultimodalPredictor,
)

To access SageMaker to train and deploy models, you need to use an IAM role that has adequate SageMaker and S3 permissions. If `role` is set to `sagemaker.get_execution_role()`, then the default role for the account profile needs to have the required permissions. If a different role is used, the `role` variable can be set to the URI of the intended IAM role. The easiest way to get all the required SageMaker permissions is to use the `AmazonSageMakerFullAccess` policy. But you should note that this is an overly permissive policy, and policies should be scoped down to the minimum required permissions necessary in non-tutorial environments.

In [ ]:
boto_session = boto3.session.Session()
sagemaker_client = boto_session.client("sagemaker")
sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_client
)

region = boto_session.region_name
role = sagemaker.get_execution_role()

bucket = sagemaker_session.default_bucket()
s3_prefix = f"autogluon_sm/{utils.sagemaker_timestamp()}"
output_path = f"s3://{bucket}/{s3_prefix}/output/"

# Retrieve Data

We will use a simplified and subsampled version of the [PetFinder dataset](https://www.kaggle.com/c/petfinder-adoption-prediction) in this tutorial. The task is to predict the animals' adoption rates based on their adoption profile information. In this simplified version, the adoption speed is grouped into two categories: 0(slow) and 1(fast).

In [ ]:
local_data_dir = "./data"
data_zip_file = "https://automl-mm-bench.s3.amazonaws.com/petfinder_for_tutorial.zip"

zip_path, _ = urllib.request.urlretrieve(data_zip_file)
with zipfile.ZipFile(zip_path, "r") as f:
    f.extractall(local_data_dir)

dataset_path = local_data_dir + "/petfinder_for_tutorial"

# Train Model On SageMaker

SageMaker provides documentation on how users can create their own training/inference scripts: [SageMaker Python SDK examples](https://sagemaker.readthedocs.io/en/stable/overview.html#prepare-a-training-script). This tutorial includes scripts that follow the official examples with a few modifications specific to AutoGluon. Here we use the [official AutoGluon Deep Learning Container images](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#autogluon-training-containers) with custom training scripts (see `scripts/` directory). The scripts created for this tutorial allow AutoGluon configuration to be passed as a YAML file (located in `/config` directory). 

In [ ]:
training_instance_type = "ml.g4dn.2xlarge"

ag = AutoGluonTraining(
    role=role,
    entry_point="scripts/multimodal_train.py",
    region=region,
    instance_count=1,
    instance_type=training_instance_type,
    framework_version="0.5.2",
    py_version="py38",
    base_job_name="autogluon-multimodal-train",
)

Create a GZIP file of training images to upload to S3 alongside the training datasets.

In [ ]:
image_tar_filename = shutil.make_archive(
    base_name="images", format="gztar", root_dir=dataset_path, base_dir="images"
)

Upload data to S3 for training. The train data, the test data, the compressed file of images, and the config file are all uploaded as separate data input channels to SageMaker.

In [ ]:
train_input = ag.sagemaker_session.upload_data(
    path=os.path.join(dataset_path, "train.csv"), key_prefix=s3_prefix
)
eval_input = ag.sagemaker_session.upload_data(
    path=os.path.join(dataset_path, "test.csv"), key_prefix=s3_prefix
)
image_input = ag.sagemaker_session.upload_data(
    path=os.path.join(".", "images.tar.gz"), key_prefix=s3_prefix
)

config_filename = "config-fast.yaml"

config_input = ag.sagemaker_session.upload_data(
    path=os.path.join(".", "config", config_filename), key_prefix=s3_prefix
)

## Fit The Model

Following the configuration and data preparation steps completed above, we can call fit on the SageMaker model.

In [ ]:
job_name = utils.unique_name_from_base("test-autogluon-image")
ag.fit(
    {"config": config_input, "train": train_input, "test": eval_input, "images": image_input},
    job_name=job_name,
)

## Model Export

AutoGluon models are portable — everything needed to deploy a trained model is in the tarball created by SageMaker. The artifact can be used locally, on EC2/ECS/EKS, or served via SageMaker Inference. Note: This will be a relatively large file because we set `standalone=True` in the training script when saving the trained `MultiModalPredictor`, which includes the `transformers.CLIPModel` and `transformers.AutoModel` in the tar file, so no online environment is needed during inference.

In [ ]:
model_s3_key = "/".join(ag.model_data.split("/")[3:])
sagemaker_session.download_data(path=".", bucket=bucket, key_prefix=model_s3_key)

# Endpoint Deployment

We can now upload the trained AutoGluon model, configure it for inference on SageMaker, and deploy it as an endpoint.

In [ ]:
endpoint_name = sagemaker.utils.unique_name_from_base("sagemaker-autogluon-serving-trained-model")

model_data = sagemaker_session.upload_data(
    path=os.path.join(".", "model.tar.gz"), key_prefix=f"{endpoint_name}/models"
)

In [ ]:
inference_instance_type = "ml.g4dn.xlarge"

model = AutoGluonInferenceModel(
    model_data=model_data,
    role=role,
    region=region,
    framework_version="0.5.2",
    py_version="py38",
    instance_type=inference_instance_type,
    source_dir="scripts",
    entry_point="multimodal_serve.py",
)

In [ ]:
predictor = model.deploy(
    initial_instance_count=1, serializer=CSVSerializer(), instance_type=inference_instance_type
)

## Predict On Unlabeled Test Data

Before using the deployed endpoint, we will create a small unlabeled dataset to perform inference on. In order to send records with their respective images to the endpoint we have to choose a serialization method for the images. Here we choose to read the images from disk into a binary buffer and encode the data into ASCII directly in the image column. In the `multimodal_serve.py` script we include the logic to reverse this process: decode the ASCII to bytes, write the bytes to disk, and replace the ASCII column with image paths, which is how AutoGluon represents image features.

In [ ]:
def path_expander(path, base_folder):
    return os.path.abspath(os.path.join(base_folder, path))


def read_image_bytes_and_encode(image_path):
    with open(image_path, "rb") as image_fo:
        image_bytes = image_fo.read()
    b64_image = base64.b85encode(image_bytes).decode("utf-8")

    return b64_image


def convert_image_path_to_encoded_bytes_in_dataframe(dataframe, image_column):
    dataframe[image_column] = [
        read_image_bytes_and_encode(path) for path in dataframe[image_column]
    ]

    return dataframe

In [ ]:
image_col = "Images"
label_col = "AdoptionSpeed"

test_data = pd.read_csv(os.path.join(dataset_path, "test.csv"), index_col=0)

test_data[image_col] = test_data[image_col].apply(
    lambda ele: path_expander(ele.split(";")[0], base_folder=dataset_path)
)  # Use only first image for tutorial

test_data_w_serialized_img = convert_image_path_to_encoded_bytes_in_dataframe(test_data, image_col)
test_data_unlabeled_raw = test_data_w_serialized_img.drop(columns=label_col)[:100].to_numpy()

In [ ]:
preds = predictor.predict(
    test_data_unlabeled_raw,
    initial_args={"Accept": "application/x-parquet", "ContentType": "text/csv"},
)

correct_preds = sum(
    pd.DataFrame(preds)[1].round().to_numpy() == test_data[label_col][: len(preds)]
) / len(preds)

print(f"{correct_preds} are correct")

## Cleanup Endpoint

Make sure to remove the inference endpoint after use because the account will be charged the associated SageMaker costs for the instance while it is `InService`.

In [ ]:
predictor.delete_endpoint()

# Batch Transform

Deploying a trained model to a hosted endpoint has been available in SageMaker since launch and is a great way to provide real-time predictions to a service like a website or mobile app. If the goal is to generate predictions from a trained model on a large dataset where minimizing latency isn’t a concern, the batch transform functionality may be more appropriate.

Read more about [Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html).

In [ ]:
endpoint_name = sagemaker.utils.unique_name_from_base(
    "sagemaker-autogluon-batch-transform-trained-model"
)

model_data = sagemaker_session.upload_data(
    path=os.path.join(".", "model.tar.gz"), key_prefix=f"{endpoint_name}/models"
)

In [ ]:
batch_inference_instance_type = "ml.g4dn.xlarge"

model = AutoGluonInferenceModel(
    model_data=model_data,
    role=role,
    region=region,
    framework_version="0.5.2",
    py_version="py38",
    instance_type=batch_inference_instance_type,
    entry_point="multimodal_serve.py",
    source_dir="scripts",
    predictor_cls=AutoGluonMultimodalPredictor,
)

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_inference_instance_type,
    strategy="MultiRecord",
    max_payload=6,
    max_concurrent_transforms=1,
    output_path=output_path,
    accept="application/json",
    assemble_with="Line",
)

Prepare batch of test data for batch transform on SageMaker

In [ ]:
test_data_for_batch_filename = "test_data_sample_no_label_no_header.csv"
batch_sample_size = 10

test_data_sample_for_batch = test_data.drop(label_col, axis=1)[:batch_sample_size]
test_data_sample_for_batch.to_csv(
    os.path.join(dataset_path, test_data_for_batch_filename), header=False, index=False
)

Upload data to S3

In [ ]:
test_input_batch = transformer.sagemaker_session.upload_data(
    path=os.path.join(dataset_path, test_data_for_batch_filename), key_prefix=s3_prefix
)

Call transform on data batch

In [ ]:
transformer.transform(
    test_input_batch,
    split_type="Line",
    content_type="text/csv",
)

transformer.wait()

Download batch transform outputs and compute scores

In [ ]:
!aws s3 cp {transformer.output_path[:-1]}/{test_data_for_batch_filename}.out .

In [ ]:
batch_predictions = pd.read_json(f"{test_data_for_batch_filename}.out")

correct_batch_preds = sum(
    batch_predictions[1].round().to_numpy() == test_data[label_col][: len(batch_predictions)]
) / len(batch_predictions)

print(f"{correct_batch_preds} are correct")

# Conclusion

In this tutorial you trained a MultiModal AutoGluon model and explored multiple to deploy it using SageMaker. Note that all the demonstrated functionalities (training, endpoint inference, batch inference) can be leveraged independently (i.e. train locally, deploy to SageMaker, or vice versa).

Next steps:
- [Learn more](https://auto.gluon.ai/) about AutoGluon
- Explore the AutoGluon [tutorials](https://auto.gluon.ai/stable/tutorials/index.html)
- Explore [SageMaker inference documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html)